<a href="https://www.kaggle.com/code/scottxchoo/2-eda-exploratory-data-analysis?scriptVersionId=144429296" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## EDA (Exploratory Data Analysis)

This notebook was created for the purpose of organizing the contents of [the book "Machine Learning Deep Learning Problem Solving Strategy"](https://goldenrabbit.co.kr/product/must-have-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%C2%B7%EB%94%A5%EB%9F%AC%EB%8B%9D-%EB%AC%B8%EC%A0%9C%ED%95%B4%EA%B2%B0-%EC%A0%84%EB%9E%B5/).

이 노트북은 [책 "머신러닝 딥러닝 문제해결 전략"](https://goldenrabbit.co.kr/product/must-have-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%C2%B7%EB%94%A5%EB%9F%AC%EB%8B%9D-%EB%AC%B8%EC%A0%9C%ED%95%B4%EA%B2%B0-%EC%A0%84%EB%9E%B5/) 내용을 정리하는 목적으로 만들어졌습니다.

.

In order to solve a problem, you must first understand what the problem is. The challenge in this competition was to predict future demand based on the historical record of a bike rental system. In order to solve this problem, you must first take a close look at the data you are given and figure out which data will and will not help you make a prediction. The step to figure this out is exploratory data analysis (EDA).

문제를 해결하려면 당연히 주어진 문제가 무엇인지부터 이해해야 합니다. 이번 경진대회의 과제는 자전거 무인 대여 시스템의 과거 기록을 기반으로 향후 수요를 예측하는 것이었습니다. 이 문제를 풀려면 우선 **주어진 데이터를 면밀히 살펴서 어느 데이터가 예측에 도움될지, 혹은 되지 않을지를 파악**해야 합니다. 이를 파악하는 단계가 바로 EDA(탐색적 데이터 분석)입니다.

## 1. Load the data (데이터 불러오기)

Let's take a look at how the given data is organized. First, we'll import the training, test, and submission sample data into Pandas as a DataFrame.

주어진 데이터가 어떻게 구성되어 있는지 살펴보겠습니다. 우선 판다스로 훈련, 테스트, 제출 샘플 데이터를 DataFrame 형태로 불러오겠습니다.

In [ ]:
import numpy as np
import pandas as pd
# 데이터 경로
data_path = '/kaggle/input/bike-sharing-demand/'

train = pd.read_csv(data_path + 'train.csv') # 훈련 데이터
test = pd.read_csv(data_path + 'test.csv') # 테스트 데이터
submission = pd.read_csv(data_path + 'sampleSubmission.csv') # 제출 샘플 데이터

Let's check the size of the training and test data with the `shape()` function.

`shape()`함수로 훈련 데이터와 테스트 데이터의 크기를 확인해보겠습니다.

In [ ]:
train.shape, test.shape

The training data consists of **10,886 rows and 12 columns** and the test data consists of **6,493 rows and 9 columns**. The number of columns represents the number of features, but why is the number of features different in the two data? Let's print the first 5 rows of the DataFrame using the `head()` function to see what feature data it contains.

훈련 데이터는 **10,886행 12열**로 구성되어 있고, 테스트 데이터는 **6,493행 9열**로 구성되어 있습니다. 열의 개수는 피처의 개수를 나타냅니다. 그런데 왜 두 데이터의 피처 개수가 다를까요? 어떤 피처 데이터를 담고 있는지 `head()`함수를 통해 DataFrame의 첫 5행을 출력해보겠습니다.

In [ ]:
train.head()

From `datetime` to `registered` are the features that can be used for prediction, and `count` is the target value to be predicted. Note that `datetime` is recorded at hourly intervals. In the end, the value we need to predict is the total number of bike rentals per hour.

`datetime`부터 `registered`까지 예측에 사용할 수 있는 피처고, `count`는 예측해야 할 타깃값입니다. `datetime`은 한 시간 간격으로 기록되어 있습니다. 결국 예측해야 할 값은 시간당 총 자전거 대여 수량입니다.

.

Let's take a look at some test data.

테스트 데이터도 한 번 보겠습니다.

In [ ]:
test.head()

The test data has fewer features than the training data. The features in the training data are missing `casual` and `registered`.

테스트 데이터는 피처 수가 훈련 데이터보다 적습니다. 훈련 데이터의 피처에서 `casual`과 `registered`가 빠졌습니다.

.

After training the model with the training data, we need to predict the number of rentals (count) with the test data, and the data we use to make the prediction is the test data. However, since the test data does not have the features `casual` and `registered`, we need to subtract the features `casual` and `registered` from the training data when training the model.

훈련 데이터를 활용해 모델을 훈련한 뒤, 테스트 데이터를 활용해 대여 수량(count)을 예측해야 되는데, 예측할 때 사용하는 데이터가 테스트 데이터입니다. 그런데 테스트 데이터에 `casual`과 `registered` 피처가 없으므로 모델을 훈련할 때도 훈련 데이터의 `casual`과 `registered` 피처를 빼야합니다.

.

Now let's see what the submission sample file looks like.

이제 제출 샘플 파일이 어떻게 생겼는지 보겠습니다.

In [ ]:
submission.head()

A submission file usually looks like this. It consists of an ID value that separates the data (in this case, datetime) and a target value. Right now, the target value, count, is all 0. You'll want to change this value to predict the number of rentals per hour and submit it. Note that the identity value (datetime) here only serves to separate the data and does not help us predict the target value, so we plan to remove the `datetime` feature from the training data when we train the model in the future.

제출 파일은 보통 이런 형태입니다. 데이터를 구분하는 ID 값(여기서는 datetime)과 타깃값으로 구성되어 있습니다. 현재는 타깃값인 count가 모두 0입니다. 시간대별 대여 수량을 예측해 이 값을 바꾼 뒤 제출하면 됩니다. 여기서 ID 값(datetime)은 데이터를 구분하는 역할만 하므로 타깃값을 예측하는 데에는 아무런 도움을 주지 않습니다. 따라서 추후 모델 훈련 시 훈련 데이터에 있는 `datetime` 피처는 제거할 계획입니다.

.

Using the info function, we can find out how many missing values there are in each column of the DataFrame and what the data type is.

info 함수를 사용하면 DataFrame 각 열의 결측값이 몇 개인지, 데이터 타입은 무엇인지 파악할 수 있습니다.

In [ ]:
train.info()

There are no missing values in the training data because the non-null count for all features is 10,886, which is the same as the total data count. If there are missing values, they need to be handled appropriately.

모든 피처의 비결측값 데이터 개수(Non-Null Count)가 전체 데이터 개수와 똑같은 10,886개이므로 훈련 데이터에는 결측값이 없습니다. 만약 결측값이 있다면 적절히 처리해줘야 합니다.

.

You can replace the missing values with the mean, median, and minimum values of the feature, or you can remove the feature containing the missing values altogether. Alternatively, you can treat the missing values as targets and use other features to predict the missing values. You can model this by thinking of the data without missing values as training data and the data with missing values as test data.

결측값을 해당 피처의 평균값, 중앙값, 최빈값으로 대체하거나 결측값을 포함하는 피처를 아예 제거하는 방법이 있습니다. 또는 결측값을 타깃값으로 간주하고, 다른 피처를 활용해 결측값을 예측할 수도 있습니다. 결측값이 없는 데이터를 훈련 데이터, 결측값이 있는 데이터를 테스트 데이터로 생각해 모델링하면 됩니다.

.

Let's take a look at the test data.

테스트 데이터도 살펴보겠습니다.

In [ ]:
test.info()

The test data also has no missing values, and the data type is the same as the training data.

테스트 데이터에도 결측값이 없고, 데이터 타입도 훈련 데이터와 동일합니다.

.

That's a quick tour of the data we'll be using in this competition.

여기까지 이번 경진대회에서 사용할 데이터의 모습을 간단히 둘러보았습니다.

## 2. Feature Engineering (피처 엔지니어링)

Once you've done some basic analysis, it's time to visualize your data. This is because visualizing your data from different perspectives can reveal trends, commonalities, and differences that are hard to find in the raw data state.

기본적인 분석을 마쳤다면 다음은 데이터 시각화 차례입니다. 데이터를 다양한 관점에서 시각화해보면 raw data 상태에서는 찾기 어려운 경향, 공통점, 차이 등이 드러날 수 있기 때문입니다.

.

However, **some data may not be in a form that lends itself to visualization**. In this contest, this is the case with the `datetime` feature. Let's analyze this feature and transform it (feature engineering) to make it suitable before visualizing it.

그런데 **일부 데이터는 시각화하기에 적합하지 않은 형태**일 수 있습니다. 본 경진대회에서는 `datetime` 피처가 그렇습니다. 시각화하기 전에 이 피처를 분석하여 적합하게 변환(피처 엔지니어링)해봅시다.

.

The data type of the `datetime` feature is object. In Pandas, the object type is a string type. The `datetime` consists of year, month, day, hour, minute, and second, so let's break it down into its components to analyze it in detail.

`datetime` 피처의 데이터 타입은 object입니다. 판다스에서 object 타입은 문자열 타입이라고 보면 됩니다. `datetime`은 연도, 월, 일, 시간, 분, 초로 구성되어 있습니다. 따라서 세부적으로 분석해보기 위해 구성요소별로 나누어보겠습니다.

.

We can easily do this with Python's built-in function `split()`, and I'll use the 100th element of `datetime` as an example to show how to do it.

파이썬 내장 함수인 `split()`을 쓰면 쉽게 나눌 수 있는데, `datetime`의 100번째 요소를 예로 들어 어떻게 나누는지 설명하겠습니다.

In [ ]:
print(train['datetime'][100]) # datetime 100번째 요소
print(train['datetime'][100].split()) # 공백 기준으로 문자열 나누기
print(train['datetime'][100].split()[0]) # 날짜
print(train['datetime'][100].split()[1]) # 시간

Because the `datetime` feature is of type object, it can be treated like a string. In the previous example, we used the `split()` function to split the leading and trailing characters based on spaces. The first string '2011-01-05' is a date string, and the second string '09:00:00' is a time string.

`datetime` 피처는 object 타입이기 때문에 문자열처럼 다룰 수 있습니다. 앞의 예에서는 `split()` 함수를 사용해 공백 기준으로 앞 뒤 문자를 나누었습니다. 첫 번째 문자열 '2011-01-05'는 날짜 문자열이고, 두 번째 문자열 '09:00:00'은 시간 문자열입니다.

.

Let's split the date string again into year, month, and day.

날짜 문자열을 다시 연도, 월, 일로 나눠보겠습니다.

In [ ]:
date = train['datetime'][100].split()[0]

print(date) # 날짜
print(date.split("-")) # "-" 기준으로 문자열 나누기
print(date.split("-")[0]) # 연도
print(date.split("-")[1]) # 월
print(date.split("-")[2]) # 일

We divided by the "-" character to get the year, month, and day.

"-" 문자를 기준으로 나누어 연도, 월, 일을 구했습니다.

.

Next, we'll divide the time string into hours, minutes, and seconds, using the ":" character is the basis for the division.

이어서 시간 문자열을 시, 분, 초로 나누겠습니다. ":" 문자가 나누는 기준입니다.

In [ ]:
time = train['datetime'][100].split()[1]

print(time) # 시간
print(time.split(":")) # ":" 기준으로 문자열 나누기
print(time.split(":")[0]) # 시
print(time.split(":")[1]) # 분
print(time.split(":")[2]) # 초

Next, we'll apply the preceding logic to `datetime` with the Pandas `apply()` function to create the date, year, month, day, hour, minute, and second features.

다음으로 판다스 `apply()` 함수로 앞의 로직을 `datetime`에 적용해 날짜(date), 연도(year), 월(month), 일(day), 시(hour), 분(minute), 초(second) 피처를 생성하겠습니다.

In [ ]:
train['date'] = train['datetime'].apply(lambda x: x.split()[0]) # 날짜 피처 생성
    
# 연도, 월, 일, 시, 분, 초 피처를 차례로 생성
train['year'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[0])
train['month'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[1])
train['day'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[2])
train['hour'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[0])
train['minute'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[1])
train['second'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[2])

The `apply()` function batches the data in a DataFrame. As you can see, it is often used in conjunction with a lambda function. It functions to apply a lambda function along the axis of the DataFrame (which by default is done for each column of the DataFrame).

`apply()` 함수는 DataFrame의 데이터를 일괄 가공해줍니다. 보다시피 종종 람다(lambda) 함수와 함께 사용됩니다. 람다 함수를 DataFrame 축(기본값은 DataFrame의 각 열에 대해 수행)을 따라 적용하는 기능을 합니다.

.

Now let's also create a day of the week feature, which can be created by utilizing the calendar and datetime 'libraries'. Where datetime is a library for manipulating dates and times, which is different from the `datetime` feature. Let's take a look at how to extract the day of the week from a date string, step by step.

이제 요일 피처도 생성해보겠습니다. 요일 피처는 calendar와 datetime '라이브러리'를 활용해 만들 수 있습니다. 여기서 datetime은 날짜와 시간을 조작하는 라이브러리로 `datetime` 피처와 다른 것입니다. 날짜 문자열에서 요일을 추출하는 방법을 한 단계씩 알아보겠습니다.

In [ ]:
from datetime import datetime
import calendar

yyyymmddDatetime = datetime.strptime(train['date'][100], '%Y-%m-%d')

print(train['date'][100]) # 날짜
print(yyyymmddDatetime) # datetime 타입으로 변경
# 정수로 요일 반환
print(yyyymmddDatetime.weekday())
# 문자열로 요일 반환
print(calendar.day_name[yyyymmddDatetime.weekday()])

It's a bit complicated, but you can use the calendar and datetime libraries to get a day of the week feature as a character. A 0 maps to Monday, a 1 to Tuesday, a 2 to Wednesday, and so on. However, you shouldn't replace the feature values with letters when training your model. Machine learning models only recognize numbers, so any character features must also be converted to numbers. Here, we've converted the day of the week feature to a string to make it easier to recognize when graphed.

다소 복잡하지만 calendar와 datetime 라이브러리를 사용하면 요일 피처를 문자로 구할 수 있습니다. 0은 월요일, 1은 화요일, 2는 수요일순으로 매핑됩니다. 단, 모델을 훈련할 때는 피처 값을 문자로 바꾸면 안 됩니다. 머신러닝 모델은 숫자만 인식하기 때문입니다. 문자 피처도 모두 숫자로 변환해야 합니다. 여기서 그래프로 나타냈을 때 쉽게 알아보려고 요일 피처를 문자열로 바꾼 겁니다.

.

We'll add the weekday feature by applying it with the preceding logic `apply()` function.

앞의 로직 `apply()` 함수로 적용해 요일(weekday) 피처를 추가하겠습니다.

In [ ]:
train['weekday'] = train['date'].apply(
    lambda dateString:
    calendar.day_name[datetime.strptime(dateString, "%Y-%m-%d").weekday()])

Next up are the `season` and `weather` features, which are categorical data, but currently represented as numbers 1, 2, 3, and 4, so it's hard to know exactly what they mean. Let's turn them into strings using the `map()` function so that their meaning is clearer when visualized.

다음은 `season`과 `weather` 피처 차례입니다. 이 두 피처는 범주형 데이터인데 현재 1, 2, 3, 4라는 숫자로 표현되어 있어서 정확히 어떤 의미인지 파악하기 어렵습니다. 시각화 시 의미가 잘 드러나도록 `map()` 함수를 사용하여 문자열로 바꾸겠습니다.

In [ ]:
train['season'] = train['season'].map({1: 'Spring',
                                       2: 'Summer',
                                       3: 'Fall',
                                       4: 'Winter'})

train['weather'] = train['weather'].map({1: 'Clear',
                                         2: 'Mist, Few clouds',
                                         3: 'Light Snow, Rain, Thunderstorm',
                                         4: 'Heavy Rain, Thunderstorm, Snow, Fog'})

In [ ]:
train.head()

1. date, year, month, day, hour, minute, second, and weekday features were added, and
2. season and weather features were changed from numbers to letters.

Note that the information provided by the `date` feature is also present in the `year`, `month`, and `day` features, so we will remove the `date` feature in the future. Also, a group of three months becomes a `season`, meaning that a granular `month` feature with three months will have the same meaning as a `season` feature. Sometimes lumping overly granular features into a larger classification improves performance, so we'll keep the `season` feature and remove the `month` feature.

1. date, year, month, day, hour, minute, second, weekday 피처가 추가되었고
2. season과 weather 피처는 숫자에서 문자로 바뀌었습니다.

참고로 `date` 피처가 제공하는 정보는 모두 `year`, `month`, `day` 피처에도 있어서 추후 `date` 피처는 제거하겠습니다. 또한 세 달씩 '월'을 묶으면 '계절'이 됩니다. 즉, 세분화된 `month` 피처를 세 달씩 묶으면 `season` 피처와 의미가 같아집니다. 지나치게 세분화된 피처를 더 큰 분류로 묶으면 성능이 좋아지는 경우가 있어서 여기서는 `season` 피처만 남기고 `month` 피처는 제거하겠습니다.

## 3. Data Visualization (데이터 시각화)

Let's visualize the feature-added training data as a graph. Visualization is the most important part of EDA. It allows us to understand the distribution of data or the relationship between data at a glance. It can also provide information that can help us with modeling.

피처를 추가한 훈련 데이터를 그래프로 시각화해보죠. 시각화는 EDA에서 가장 중요한 부분입니다. 데이터 분포나 데이터 간 관계를 한눈에 파악할 수 있기 때문입니다. 모델링에 도움될 만한 정보를 얻을 수도 있죠.

.

We'll use the matplotlib and seaborn libraries for visualization. matplotlib is a standard library for visualizing data in Python, and seaborn is a library that adds a high-level interface to matplotlib.

시각화를 위해 matplotlib과 seaborn 라이브러리를 활용하겠습니다. matplolib은 파이썬으로 데이터를 시각화할 때 표준처럼 사용되는 라이브러리이며, seaborn은 matplotlib에 고수준 인터페이스를 덧씌운 라이브러리입니다.

.

First, import the two libraries.

먼저 두 라이브러리를 임포트합니다.

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
# matplotlib이 그린 그래프를 주피터 노트북에서 바로 출력해준다.
%matplotlib inline

In the following sections, we'll use these libraries to plot various graphs.

이제부터 이 라이브러리들을 활용하여 각종 그래프를 그려보겠습니다.

### [1] Distribution plot (분포도)

A distribution plot is a graph that shows the aggregate value of numerical data. An aggregate value is a total count or a percentage, for example. In this chapter, we'll plot a distribution plot of COUNT because knowing the distribution of the target value will help us know whether to use the target value as is or transform it for training.

분포도(distribution plot)는 수치형 데이터의 집계 값을 나타내는 그래프입니다. 집계 값은 총 개수나 비율 등을 의미합니다. 타깃값은 count의 분포도를 그려보겠습니다. 이번 장에서는 타깃값의 분포를 알면 훈련 시 타깃값을 그대로 사용할지 변환해 사용할지 파악할 수 있기 때문입니다.

In [ ]:
mpl.rc('font', size=15) # 폰트 크기를 15로 설정
sns.displot(train['count']) # 분포도 출력

The x-axis represents the target value, count, and the y-axis represents the total number of counts. The distribution shows that the target value, count, is clustered around 0. This means that the distribution is heavily skewed to the left. For a regression model to perform well, the data must be normally distributed, and the current target value, count, is not normally distributed. Therefore, modeling with the current target value is unlikely to perform well.

x축은 타깃값인 count를 나타내고, y축은 총 개수를 나타냅니다. 분포도를 보면 타깃값인 count가 0 근처에 몰려 있습니다. 즉, 분포가 왼쪽으로 많이 편향되어 있습니다. 회귀 모델이 좋은 성능을 내려면 데이터가 정규분포를 따라야 하는데, 현재 타깃값 count는 정규분포를 따르지 않습니다. 따라서 현재 타깃값을 그대로 사용해 모델링한다면 좋은 성능을 기대하기 어렵습니다.

.

The most common way to make data distributions more normal is to log-transform them. Logarithmic transformation is used when data is skewed to the left, such as a count distribution. Logarithmic transformation is simple. Just take the logarithm of the value you want. Let's look at the distribution of count log-transformed.

데이터 분포를 정규분포에 가깝게 만들기 위해 가장 많이 사용하는 방법은 로그변환입니다. 로그변환은 count 분포와 같이 데이터가 왼쪽으로 편향되어 있을 때 사용합니다. 로그변환하는 방법은 간단합니다. 원하는 값에 로그를 취해주면 됩니다. count를 로그변환한 값의 분포를 살펴보겠습니다.

In [ ]:
sns.displot(np.log(train['count']))

It is closer to a normal distribution than before the transformation. We said that the closer the target value distribution is to a normal distribution, the better the regression model performs. In other words, it's more accurate to predict log(count) than to predict count directly from the feature. So we'll convert our target value to log(count).

변환 전보다 정규분포에 가까워졌습니다. 타깃값 분포가 정규분포에 가까울수록 회귀 모델 성능이 좋다고 했습니다. 다시 말해, 피처를 바로 활용해 count를 예측하는 것보다 log(count)를 예측하는 편이 더 정확합니다. 따라서 우리도 타깃값을 log(count)로 변환해 사용하겠습니다.

.

However, we need to do an exponential conversion at the end to restore the actual target value, count. Exponentiating log(y) gives us y, as shown in the following formula.

다만, 마지막에 지수변환을 하여 실제 타깃값인 count로 복원해야 합니다. 다음 수식이 나타내는 바와 같이 log(y)를 지수변환하면 y가 됩니다.

### [2] Bar graph (막대 그래프)

Next, we'll plot a bar graph of the average number of rentals for each of the six features: year, month, day, hour, minute, and second. These features are categorical data. We want to understand how the average number of rentals varies for each categorical data, so we know which features are important. This is where a bar graph comes in. Bar graphs can be plotted with seaborn's barplot() function.

다음으로 year, month, day, hour, minute, second 별로 총 여섯 가지의 평균 대여 수량을 막대 그래프로 그려보겠습니다. 이 피처들은 범주형 데이터입니다. 각 범주형 데이터에 따라 평균 대여 수량이 어떻게 다른지 파악하려고 합니다. 그래야 어떤 피처가 중요한지 알 수 있습니다. 이럴 때 막대 그래프를 이용합니다. 막대 그래프는 seaborn의 barplot() 함수로 그릴 수 있습니다.

In [ ]:
# Step 1 : m행 n열 Figure 준비
mpl.rc('font', size=14)                       # 폰트 크기 설정
mpl.rc('axes', titlesize=15)                  # 각 축의 제목 크기 설정
figure, axes = plt.subplots(nrows=3, ncols=2) # 3행 2열 Figure 생성
plt.tight_layout()                            # 그래프 사이에 여백 확보
figure.set_size_inches(10, 9)                 # 전체 Figure 크기를 10x9인치로 설정

# Step 2 : 각 축에 서브플롯 할당
## 각 축에 연도, 월, 일, 시간, 분, 초별 평균 대여 수량 막대 그래프 할당
sns.barplot(x='year', y='count', data=train, ax=axes[0, 0])
sns.barplot(x='month', y='count', data=train, ax=axes[0, 1])
sns.barplot(x='day', y='count', data=train, ax=axes[1, 0])
sns.barplot(x='hour', y='count', data=train, ax=axes[1, 1])
sns.barplot(x='minute', y='count', data=train, ax=axes[2, 0])
sns.barplot(x='second', y='count', data=train, ax=axes[2, 1])

# Step 3 : 세부 설정
## 3-1 : 서브플롯에 제목 달기
axes[0, 0].set(title='Rental amounts by year')
axes[0, 1].set(title='Rental amounts by month')
axes[1, 0].set(title='Rental amounts by day')
axes[1, 1].set(title='Rental amounts by hour')
axes[2, 0].set(title='Rental amounts by minute')
axes[2, 1].set(title='Rental amounts by second')
## 3-2 : 1행에 위치한 서브프롯들의 x축 라벨 90도 회전
axes[1, 0].tick_params(axis='x', labelrotation=90)
axes[1, 1].tick_params(axis='x', labelrotation=90)

Let's take a quick look at the six graphs and see what they tell us.

**1. 'Average rentals by year' :** There were more rentals in 2012 than in 2011.

**2. 'Average rentals by month' :** The average number of rentals is highest in June and lowest in January. You can guess that the warmer the weather, the higher the number of rentals.

**3. 'Average rental quantity per day' :** There is no obvious difference in the rental quantity per day. As I said in the introduction page, the training data only has data from the 1st to the 19th of each month. The rest of the data from the 20th to the end of the month is in the test data. So we can't use day as a feature, because day in the training data and day in the test data have completely different values.

**4. 'Average number of rentals per hour' :** The graph shape is a bell curve. The lowest number of rentals is at 4 a.m., which makes sense, since few people ride bikes at 4 a.m. On the other hand, the highest number of rentals is at 8 a.m. and 5-6 p.m. We can assume that people use bikes a lot on their way to school or work.

**5. 'Average number of rentals per minute and second' :** It doesn't contain any information, so we won't use the minute and second features when we train the model later.

여섯 개의 그래프를 보면서 어떤 정보를 담고 있는지 가볍게 훑어봅시다.

**1. '연도별 평균 대여 수량' :** 2011년보다 2012년에 대여가 더 많았네요.

**2. '월별 평균 대여 수량' :** 평균 대여 수량은 6월에 가장 많고 1월에 가장 적습니다. 날씨가 따뜻할수록 대여 수량이 많다고 짐작할 수 있습니다.

**3. '일별 평균 대여 수량' :** 일별 대여 수량에는 뚜렷한 차이가 없습니다. 소개 페이지에서 말했다시피 훈련 데이터에는 매월 1일부터 19일까지의 데이터만 있습니다. 나머지 20일부터 월말까지의 데이터는 테스트 데이터에 있습니다. 그래서 일자(day)는 피처로 사용하지 못합니다. 왜냐하면 훈련 데이터의 day와 테스트 데이터의 day는 전혀 다른 값을 갖고 있기 때문입니다.

**4. '시간별 평균 대여 수량' :** 그래프 모양이 쌍봉형입니다. 새벽 4시에 대여 수량이 가장 적습니다. 당연하겠죠. 새벽 4시에 자전거를 타는 사람은 거의 없을 테니까요. 반면 아침 8시와 저녁 5~6시에 대여가 가장 많습니다. 사람들이 등하교 혹은 출퇴근 길에 자전거를 많이 이용한다고 짐작해볼 수 있습니다.

**5. '분별, 초별 평균 대여 수량' :** 아무 정보도 담지 않고 있습니다. 따라서 나중에 모델을 훈련할 때 분과 초 피처는 사용하지 않겠습니다.

### [3] Box plot (박스플롯)
A box plot is a graph that represents numerical data information based on categorical data. It is characterized by providing more information than a bar graph.

In this example, we'll boxplot the number of rentals (numerical data) by season, weather, holiday, and workday (categorical data). You can see how our target value, the number of rentals, changes based on each categorical data.

박스플롯(box plot)은 범주형 데이터에 따른 수치형 데이터 정보를 나타내는 그래프입니다. 막대 그래프보다 더 많은 정보를 제공하는 특징이 있습니다.

여기서는 season, weather, holiday, workingday(범주형 데이터)별 대여 수량(수치형 데이터)을 박스플롯으로 그려보겠습니다. 각 범주형 데이터에 따라 타깃값인 대여 수량이 어떻게 변하는지 알 수 있습니다.

In [ ]:
# 스텝 1 : m행 n열 Figure 준비
figure, axes = plt.subplots(nrows=2, ncols=2) # 2행 2열
plt.tight_layout()
figure.set_size_inches(10, 13)

# 스텝 2 : 서브플롯 할당
## 계절, 날씨, 공휴일, 근무일별 대여 수량 박스플롯
sns.boxplot(x='season', y='count', data=train, ax=axes[0, 0])
sns.boxplot(x='weather', y='count', data=train, ax=axes[0, 1])
sns.boxplot(x='holiday', y='count', data=train, ax=axes[1, 0])
sns.boxplot(x='workingday', y='count', data=train, ax=axes[1, 1])

## 스텝 3 : 세부 설정
## 3-1 : 서브플롯에 제목 달기
axes[0, 0].set(title='Box Plot On Count Across Season')
axes[0, 1].set(title='Box Plot On Count Across Weather')
axes[1, 0].set(title='Box Plot On Count Across Holiday')
axes[1, 1].set(title='Box Plot On Count Across Working Day')
## 3-2 : x축 라벨 겹침 해결
axes[0, 1].tick_params(axis='x', labelrotation=10) # 10도 회전

1. the number of bicycle rentals is lowest in spring and highest in fall.

2. The boxplot shows the number of rentals by weather, which matches our intuition: when the weather is good, the number of rentals is the highest, and when the weather is bad, the number of rentals is low. There are almost no rentals during heavy rain, heavy snow (rightmost box in the graph).

3. A boxplot showing the number of rentals based on whether or not there is a public holiday. The x-axis label 0 means it's not a holiday, and 1 means it is. The median number of bikes rented on and off holidays is almost identical. However, there are many outliers on non-holidays.

4. shows the number of rentals based on whether it is a working day or not, with more outliers on working days. Note that a working day is defined as any day excluding holidays and weekends.

1. 자전가 대여 수량은 봄에 가장 적고, 가을에 가장 많습니다.

2. 박스플롯이 보여주는 날씨별 대여 수량은 우리의 직관과 일치합니다. 날씨가 좋을 때 대여 수량이 가장 많고, 안 좋을수록 수량이 적습니다. 폭우, 폭설이 내리는 날씨(그래프의 가장 오른쪽 박스)에는 대여 수량이 거의 없습니다.

3. 공휴일 여부에 따른 대여 수량을 나타내는 박스플롯입니다. x축 라벨 0은 공휴일이 아니라는 뜻이고, 1은 공휴일이라는 뜻입니다. 공휴일일 때와 아닐 때 자전거 대여 수량의 중앙값은 거의 비슷합니다. 다만, 공휴일이 아닐 때는 이상치(outlier)가 많습니다.

4. 근무일 여부에 따른 대여 수량을 나타내는데, 근무일일 때 이상치가 많습니다. 참고로 근무일은 공휴일과 주말을 뺀 나머지 날을 뜻합니다.

### [4] Point plot (포인트플롯)

Next, let's plot the average number of rentals per hour by workday, holiday, weekday, season, and weather as a point plot. A point plot shows the mean and confidence intervals of numerical data based on categorical data as dots and lines. It provides the same information as a bar graph, but is better suited for plotting multiple graphs on one screen to compare them to each other.

다음으로 workingday, holiday, weekday, season, weather에 따른 시간대별 평균 대여 수량을 포인트플롯으로 그려보겠습니다. 포인트플롯은 범주형 데이터에 따른 수치형 데이터의 평균과 신뢰구간을 점과 선으로 표시합니다. 막대 그래프와 동일한 정보를 제공하지만, 한 화면에 여러 그래프를 그려 서로 비교해보기에 더 적합합니다.

In [ ]:
# 스텝 1 : m행과 n열 Figure 준비
mpl.rc('font', size = 11)
figure, axes = plt. subplots(nrows = 5) # 5행 1열
figure.set_size_inches(12, 18)

# 스텝 2 : 서브플롯 할당
## 근무일, 공휴일, 요일, 계절, 날씨에 따른 시간대별 평균 대여 수량을 포인트플롯
sns.pointplot(x = 'hour', y = 'count', data = train, hue = 'workingday', ax = axes[0])
sns.pointplot(x = 'hour', y = 'count', data = train, hue = 'holiday', ax = axes[1])
sns.pointplot(x = 'hour', y = 'count', data = train, hue = 'weekday', ax = axes[2])
sns.pointplot(x = 'hour', y = 'count', data = train, hue = 'season', ax = axes[3])
sns.pointplot(x = 'hour', y = 'count', data = train, hue = 'weather', ax = axes[4])

1. on working days, the number of rentals is highest during rush hour, and on days off, the number of rentals is highest from 12-2pm.

2, 3. The point plots by day of the week, whether it is a holiday or not, are similar to the point plots by working day.

4. Looking at the point plots by season and time of day, the number of rentals is highest in the fall and lowest in the spring.

5. As expected, the number of rentals is highest when the weather is good, but there are a few rentals at 18:00 during heavy rain and snow. You might want to consider removing these outliers.

1. 근무일에는 출퇴근 시간에 대여 수량이 많고 쉬는 날에는 오후 12~2시에 가장 많습니다.

2, 3. 공휴일 여부, 요일에 따른 포인트플롯도 근무일 여부에 따른 포인트플롯과 비슷한 양상을 보입니다.

4. 계절에 따른 시간대별 포인트플롯을 보면, 대여 수량은 가을에 가장 많고, 봄에 가장 적습니다.

5. 예상대로 날씨가 좋을 때 대여량이 가장 많습니다. 그런데 폭우, 폭설이 내릴 때 18시에 대여 건수가 조금 있습니다. 이런 이상치는 제거를 고려해보는 것도 괜찮은 방법입니다.

### [5] Scatter plot with regression lines (회귀선을 포함한 산점도 그래프)

Let's plot the numerical data TEMP, ATEMP, WIND SPEED, and HUMIDITY by rental quantity as a "scatter plot with regression lines." Scatter plots with regression lines are used to identify correlations between numerical data.

수치형 데이터인 temp, atemp, windspeed, humidity별 대여 수량을 '회귀선을 포함한 산점도 그래프'로 그려보겠습니다. 회귀선을 포함한 산점도 그래프는 수치형 데이터 간 상관관계를 파악하는 데 사용합니다.

.

This graph can be plotted with seaborn's regplot() function.

이 그래프는 seaborn의 regplot() 함수로 그릴 수 있습니다.

In [ ]:
# 스텝 1 : m형 n열 Figure 준비
mpl.rc('font', size = 15)
figure, axes = plt.subplots(nrows = 2, ncols = 2) # 2행 2열
plt.tight_layout()
figure.set_size_inches(7, 6)

# 스텝 2 : 서브플롯 할당
## 온도, 체감 온도, 풍속, 습도 별 대여 수량 산점도 그래프
sns.regplot(x = 'temp', y = 'count', data = train, ax = axes[0, 0],
           scatter_kws = {'alpha': 0.3}, line_kws = {'color': 'blue'})
sns.regplot(x = 'atemp', y = 'count', data = train, ax = axes[0, 1],
           scatter_kws = {'alpha': 0.3}, line_kws = {'color': 'blue'})
sns.regplot(x = 'windspeed', y = 'count', data = train, ax = axes[1, 0],
           scatter_kws = {'alpha': 0.3}, line_kws = {'color': 'blue'})
sns.regplot(x = 'humidity', y = 'count', data = train, ax = axes[1, 1],
           scatter_kws = {'alpha': 0.3}, line_kws = {'color': 'blue'})

The slope of the regression line gives you a rough idea of the trend.

1, 2. The higher the temperature, the more rentals.

4. The lower the humidity, the more rentals. In other words, there are more rentals when it's warm than when it's cold, and more when it's not humid than when it's humid.

3. The regression line shows that the higher the wind speed, the higher the number of rentals. This is a bit odd, as I would expect the weaker the wind, the more... The reason is that there are a lot of missing values in the `windspeed` feature. If you look closely, there is quite a bit of data with a wind speed of 0. It is likely that the actual wind speed is not zero, but was recorded as zero due to no observation or error. The number of missing values makes it difficult to correlate the wind speed with the number of rentals based on the graph. The number of missing values makes it difficult to correlate the wind speed with the number of rentals based on the graph. Data with a lot of missing values should be handled appropriately: replace the missing values with other values, or delete the `windspeed` feature itself.

회귀선 기울기로 대략적인 추세를 파악할 수 있습니다.

1, 2. 온도와 체감 온도가 높을수록 대여 수량이 많습니다.

4. 습도는 낮을수록 대여를 많이 합니다. 다시 말해 대여 수량은 추울 때보다 따뜻할 때 많고, 습할 때보다 습하지 않을 때 많습니다.

3. 회귀선을 보면 풍속이 셀수록 대여 수량이 많습니다. 바람이 약할수록 많을 것 같은데 조금 이상하네요... 이유는 `windspeed` 피처에 결측값이 많기 때문입니다. 자세히 보면 풍속이 0인 데이터가 꽤 많습니다. 실제 풍속이 0이 아니라 관측치가 없거나 오류로 인해 0으로 기록됐을 가능성이 높습니다. 결측값이 많아서 그래프만으로 풍속과 대여 수량의 상관관계를 파악하기는 힘듭니다. 결측값이 많아서 그래프만으로 풍속과 대여 수량의 상관관계를 파악하기는 힘듭니다. 결측값이 많은 데이터는 적절히 처리해야 합니다. 결측값을 다른 값으로 대체하거나 `windspeed` 피처 자체를 삭제하면 됩니다.

### [6] Heatmap (히트맵)

temp, atemp, humidity, windspeed, and count are numerical data. Let's see how they correlate. The corr() function computes the correlation coefficients between the features in the DataFrame and returns a "correlation matrix between numerical data".

temp, atemp, humidity, windspeed, count는 수치형 데이터입니다. 수치형 데이터끼리 어떤 상관관계가 있는지 알아보겠습니다. corr() 함수는 DataFrame 내의 피처 간 상관계수를 계산해 "수치형 데이터 간 상관관계 매트릭스"를 반환합니다.

In [ ]:
train[['temp', 'atemp', 'humidity', 'windspeed', 'count']].corr()

But with so many combinations, it's hard to see which features are strongly related at a glance. That's where heatmaps come in. A heatmap is a colorful representation of the relationships between data, making it easy to compare multiple pieces of data at a glance. Heatmaps can be drawn with the heatmap() function in seaborn().

하지만 조합이 많아 어느 피처들의 관계가 깊은지 한눈에 들어오지 않습니다. 히트맵이 필요한 순간입니다. 히트맵은 데이터 간 관계를 색상으로 표현하여, 여러 데이터를 한눈에 비교하기에 좋습니다. 히트맵은 seaborn()의 heatmap() 함수로 그릴 수 있습니다.

In [ ]:
# 피처 간 상관관계 매트릭스
corrMat = train[['temp', 'atemp', 'humidity', 'windspeed', 'count']].corr()
fig, ax = plt.subplots()
fig.set_size_inches(8, 8)
sns.heatmap(corrMat, annot = True) # 상관관계 히트맵 그리기
ax.set(title = "Heatmap of Numerical Data")

The correlation coefficient between temp & atemp and the number of rentals (count) is 0.39. This is a positive correlation, meaning that the higher the temperature, the higher the number of rentals. On the other hand, the correlation between humidity and rentals is negative, meaning that the "lower" the humidity, the more rentals. This is the same as what we analyzed in the scatterplot graph earlier.

temp & atemp와 대여 수량(count) 간 상관계수는 0.39입니다. 양의 상관관계를 보이는군요. 온도가 높을수록 대여 수량이 많다는 뜻입니다. 반면, humidity와 대여 수량은 음수이니 습도가 '낮을수록' 대여 수량이 많다는 뜻입니다. 앞서 산점도 그래프에서 분석한 내용과 동일합니다.

.

The correlation between windspeed and rentals is 0.1. Since the correlation is very weak, we'll remove the windspeed feature.

windspeed과 대여 수량의 상관계수는 0.1입니다. 상관관계가 매우 약하므로 windspeed 피처는 제거하겠습니다.

## 4. Analysis Summary & Modeling Strategy

### Analysis Summary

So far, we've looked at the data from a variety of perspectives. Let's summarize the key takeaways from our analysis

**1. Transform the target value
   - After checking the distribution, we found that the target value, `count`, is skewed near zero, so we need to logarithmize it to make it more normally distributed. We will convert the target value to log(count) instead of count, and we need to exponentialize it back to count at the end.

**2. Add a derived feature
   - Since the `datetime` feature is a mixture of different pieces of information, we can separate them to create `year`, `month`, `day`, `hour`, `minute`, and `second` features.
   - We'll add a weekday feature, which is another piece of information hidden in the `datetime` feature.


**3. Remove derived features
   - There's no point in using features that aren't in the test data for training, so let's remove the features casual and registered, which are only in the training data.
   - The `datetime` feature only acts as an index, so it doesn't help us predict the target value.
   - The information provided by the `date` feature is in the newly added features `year`, `month`, and `day`, so we will remove them.
   - The `month` feature can be viewed as a subdivision of the `season` feature. If the data is too fine-grained, there will be less data per classification, which will actually hinder learning.
   - Inspect the bar graph and remove the derived feature `day` because it is not discriminative and `minute` and `second` contain no information.
   - After checking the scatter plot and heatmap, the feature `windspeed` has many missing values and a very weak correlation with the number of rentals.
   
**4. Remove outliers
   - Checking the point plot, the data with `weather` = 4 is an outlier.

지금까지 다양한 측면에서 데이터를 살펴보았습니다. 분석 과정에서 파악한 주요 내용을 정리해보겠습니다

**1. 타깃값 변환**
   - 분포도 확인 결과 타깃값인 `count`가 0 근처로 치우쳐 있으므로 로그변환하여 정규분포에 가깝게 만들어야 합니다. 타깃값을 count가 아닌 log(count)로 변환해 사용할 것이고 마지막에 다시 지수변환해 count로 복원해야 합니다.

**2. 파생 피처 추가**
   - `datetime` 피처는 여러 가지 정보의 혼합체이므로 각각을 분리해 `year`, `month`, `day`, `hour`, `minute`, `second` 피처를 생성할 수 있습니다.
   - `datetime` 피처에 숨어 있는 또 다른 정보인 요일(weekday) 피처를 추가하겠습니다.


**3. 파생 피처 제거**
   - 테스트 데이터에 없는 피처는 훈련에 사용해도 큰 의미가 없습니다. 따라서 훈련 데이터에만 있는 casual과 registered 피처는 제거하겠습니다.
   - `datetime` 피처는 인덱스 역할만 하므로 타깃값 예측에 아무런 도움이 되지 않습니다.
   - `date` 피처가 제공하는 정보는 새로 추가한 `year`, `month`, `day` 피처에 있으니 제거합니다.
   - `month` 피처는 `season` 피처의 세부 분류로 볼 수 있습니다. 데이터가 지나치게 세분화되어 있으면 분류별 데이터 수가 적어서 오히려 학습에 방해가 됩니다.
   - 막대 그래프 확인 결과 파생 피처인 `day`는 분별력이 없고 `minute`와 `second`에는 아무런 정보가 담겨 있지 않아 제거합니다.
   - 산점도 그래프와 히트맵 확인 결과 `windspeed` 피처에는 결측값이 많고 대여 수량과의 상관관계가 매우 약합니다.
   
**4. 이상치 제거**
   - 포인트 플롯 확인 결과 `weather`가 4인 데이터는 이상치입니다.


### Modeling Strategy

If you want to do well in the competition, you'll need to come up with your own optimized model. However, since Chapter 6 is more of a warm-up to get acquainted with Cagle, we're going to use the default models provided by CycleRun. You'll need to make sure that your model is at least as good as the default models in the future to make it worthwhile to participate in the competition, so take this opportunity to get acquainted.

- Baseline model: adopted the most basic model, LinearRegression
- Performance improvements: Ridge, Lasseau, and Random Forest regression models
   - Feature engineering: do the same for all models with the previous level of analysis
   - Hyperparameter optimization: GridSearch
- Miscellaneous: Target value is log(count) instead of count

경진대회에서 우수한 성적을 거두려면 본인만의 최적화된 모델을 구상해야 합니다. 하지만 6장은 캐글과 친해지기 위한 몸풀기 목적이 강하므로 사이킷런이 제공하는 기본 모델들만 사용하기로 했습니다. 차후 자신이 만든 모델이 최소한 기본 모델들보다는 우수해야 대회에 참여한 의의가 있을 테니 이번 기회에 친해지길 바랍니다.

- 베이스라인 모델 : 가장 기본적인 모델인 LinearRegression 채택
- 성능 개선 : 릿지, 라쏘, 랜덤 포레스트 회귀 모델
   - 피처 엔지니어링 : 앞의 분석 수준으로 모든 모델에서 동일하게 수행
   - 하이퍼파라미터 최적화 : 그리드서치
- 기타 : 타깃값이 count가 아닌 log(count)임